In [150]:
from ast import literal_eval
import glob
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Point

In [127]:
# load in crime data
rep_crime = pd.read_csv('data/crimes.csv')
# drop where no location data available
rep_crime = rep_crime[rep_crime['Latitude'].notnull()]
# now let's make it into a geodataframe
rep_crime['geometry'] = list(zip(rep_crime.Longitude, rep_crime.Latitude))
rep_crime['geometry'] = rep_crime['geometry'].apply(Point)
crime_gdf = gpd.GeoDataFrame(rep_crime, geometry='geometry')
crime_gdf.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,geometry
1,10208778,HY395790,12/31/2014 12:01:00 AM,046XX N PAULINA ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,3.0,11,1164393.0,1930830.0,2014,02/10/2018 03:50:01 PM,41.965817,-87.670943,"(41.965817209, -87.67094261)",POINT (-87.67094261 41.965817209)
2,10905004,JA217031,12/31/2014 12:01:00 AM,014XX N RIDGEWAY AVE,1751,OFFENSE INVOLVING CHILDREN,CRIM SEX ABUSE BY FAM MEMBER,RESIDENCE,False,False,...,23.0,20,1151106.0,1909346.0,2014,02/10/2018 03:50:01 PM,41.907135,-87.720361,"(41.907134847, -87.720361266)",POINT (-87.720361266 41.90713484699999)
3,9912248,HY101254,12/31/2014 12:01:00 AM,061XX S TROY ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,66.0,14,1156441.0,1863663.0,2014,02/10/2018 03:50:01 PM,41.781669,-87.701997,"(41.781669121, -87.701997046)",POINT (-87.701997046 41.781669121)
4,9913307,HY101788,12/31/2014 12:01:00 AM,029XX W WARREN BLVD,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,APARTMENT,False,False,...,27.0,11,1156720.0,1900211.0,2014,02/10/2018 03:50:01 PM,41.881956,-87.699986,"(41.881955638, -87.699986249)",POINT (-87.69998624899999 41.881955638)
5,9920890,HY108795,12/31/2014 12:01:00 AM,007XX N WABASH AVE,0820,THEFT,$500 AND UNDER,APARTMENT,False,False,...,8.0,06,1176598.0,1905516.0,2014,02/10/2018 03:50:01 PM,41.896087,-87.626835,"(41.89608709, -87.626834633)",POINT (-87.62683463299999 41.89608709)


In [97]:
# load in community area data
response = requests.get('https://data.cityofchicago.org/resource/igwz-8jzy.json').json()
comm_area_name_df = pd.DataFrame.from_dict(response)
comm_area_name_df['community_area'] = comm_area_name_df['area_num_1']
# make inot a geodataframe
comm_area_name_df['geometry'] = comm_area_name_df['the_geom'].apply(shape)
comm_area_gdf = gpd.GeoDataFrame(comm_area_name_df, geometry='geometry')
comm_area_gdf.head()

,the_geom,perimeter,area,comarea,comarea_id,area_numbe,community,area_num_1,shape_area,shape_len,community_area,geometry
0,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",0,0,0,0,35,DOUGLAS,35,46004621.1581,31027.0545098,35,(POLYGON ((-87.60914087617894 41.8446925026539...
1,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",0,0,0,0,36,OAKLAND,36,16913961.0408,19565.5061533,36,(POLYGON ((-87.59215283879394 41.8169293462668...
2,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",0,0,0,0,37,FULLER PARK,37,19916704.8692,25339.0897503,37,(POLYGON ((-87.62879823733725 41.8018930336891...
3,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",0,0,0,0,38,GRAND BOULEVARD,38,48492503.1554,28196.8371573,38,(POLYGON ((-87.6067081256125 41.81681377057218...
4,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",0,0,0,0,39,KENWOOD,39,29071741.9283,23325.1679062,39,(POLYGON ((-87.59215283879394 41.8169293462668...


In [128]:
# merge crime and community area w/ spatial join
crime_comm_df = gpd.sjoin(crime_gdf, comm_area_gdf, how="left", op='intersects')
crime_comm_df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,perimeter,area,comarea,comarea_id,area_numbe,community,area_num_1,shape_area,shape_len,community_area
1,10208778,HY395790,12/31/2014 12:01:00 AM,046XX N PAULINA ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,0,0,0,0,3,UPTOWN,3,65095642.7289,46972.7945549,3
2,10905004,JA217031,12/31/2014 12:01:00 AM,014XX N RIDGEWAY AVE,1751,OFFENSE INVOLVING CHILDREN,CRIM SEX ABUSE BY FAM MEMBER,RESIDENCE,False,False,...,0,0,0,0,23,HUMBOLDT PARK,23,100480876.502,48583.0643299,23
3,9912248,HY101254,12/31/2014 12:01:00 AM,061XX S TROY ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,0,0,0,0,66,CHICAGO LAWN,66,98279465.1151,40073.0998382,66
4,9913307,HY101788,12/31/2014 12:01:00 AM,029XX W WARREN BLVD,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,APARTMENT,False,False,...,0,0,0,0,27,EAST GARFIELD PARK,27,53883220.8462,31514.6259565,27
5,9920890,HY108795,12/31/2014 12:01:00 AM,007XX N WABASH AVE,0820,THEFT,$500 AND UNDER,APARTMENT,False,False,...,0,0,0,0,8,NEAR NORTH SIDE,8,76675895.9728,57293.1649516,8


In [136]:
crime_by_date = crime_comm_df.groupby(['Date', 'community_area', 'community'])['ID'].count().reset_index()
crime_by_date['year'] = pd.DatetimeIndex(crime_by_date['Date']).year

In [148]:
crime_by_year = crime_by_date.groupby(['year', 'community_area', 'community'])["ID"].count().reset_index()
crime_by_year = crime_by_year[crime_by_year['year'].isin(list(range(2015, 2020)))]
crime_by_year.rename(columns={'ID': 'crime_count'}, inplace=True)
crime_by_year.head()

,year,community_area,community,crime_count
72,2015,1,ROGERS PARK,3196
73,2015,10,NORWOOD PARK,1193
74,2015,11,JEFFERSON PARK,1100
75,2015,12,FOREST GLEN,425
76,2015,13,NORTH PARK,809


In [149]:
def swap_points(tup):
    '''
    Helper function to swap lat/long points for library data
    for merging w/ community area data
    '''
    tup = literal_eval(tup)
    return (tup[1], tup[0])

In [211]:
# this is for clean merger
replace_dict = {'Harold Washtington Library Center': 'Harold Washington Library Center',
                'Whitney M. Young, Jr.': 'Whitney M. Young, Jr', 'Legler Regional': 'Legler',
                'Daley, Richard J.-Bridgeport':'Daley, Richard J.',
                'Daley, Richard M.-W Humboldt':'Daley, Richard M.', 'Roosevelt': 'Little Italy',
                'Sulzer Regional Library': 'Sulzer Regional', 'Daley, Richard J': 'Daley, Richard J.',
                'Daley, Richard M': 'Daley, Richard M.', 'Woodson Regional Library':'Woodson Regional'}

In [223]:
# now let's load in our lib data
lib_files = glob.glob('data/library_visits/*.csv')
to_concat = []
for file in lib_files:
    tmp_df = pd.read_csv(file)
    tmp_df['year'] = file[-8:-4]
    to_concat.append(tmp_df)
lib_df = pd.concat(to_concat)
lib_df['branch_name'] = lib_df['LOCATION'].str.extract(r'(.*\w)')
lib_df = lib_df[lib_df['branch_name'].notnull()]
lib_df.drop(columns=['LOCATION'], inplace=True)
lib_df.replace(replace_dict, inplace=True)
lib_df.head()

/Users/katykoeing/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,ADDRESS,APRIL,AUGUST,CITY,DECEMBER,FEBRUARY,JANUARY,JULY,JUNE,MARCH,MAY,NOVEMBER,OCTOBER,SEPTEMBER,YTD,ZIP,year,branch_name
0,NaN,9436.0,11349.0,NaN,8575.0,9331.0,10351.0,9842.0,9927.0,10863.0,9325.0,10322.0,11082.0,9482.0,119885.0,NaN,2017,Lincoln Park
1,NaN,4602.0,6041.0,NaN,4950.0,4858.0,5008.0,6127.0,5987.0,4965.0,4946.0,2518.0,3349.0,5223.0,58574.0,NaN,2017,Kelly
2,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2017,Independence
3,NaN,5293.0,6733.0,NaN,4444.0,5861.0,5160.0,6162.0,6221.0,5621.0,5778.0,5283.0,5978.0,5537.0,68071.0,NaN,2017,King
4,NaN,7495.0,8779.0,NaN,6958.0,7174.0,8181.0,7983.0,8318.0,7382.0,7324.0,8656.0,8519.0,8253.0,95022.0,NaN,2017,Coleman


In [225]:
# now let's load in our lib location data
lib_locations = pd.read_csv('data/libraries_locations.csv')
lib_locations.replace(replace_dict, inplace=True)
# merge two lib dfs to we have location data for each point in each year
lib_data = pd.merge(lib_df, lib_locations, how='left', left_on='branch_name', right_on='NAME ')
lib_data.head()

,ADDRESS_x,APRIL,AUGUST,CITY_x,DECEMBER,FEBRUARY,JANUARY,JULY,JUNE,MARCH,...,branch_name,NAME,HOURS OF OPERATION,ADDRESS_y,CITY_y,STATE,ZIP_y,PHONE,WEBSITE,LOCATION
0,NaN,9436.0,11349.0,NaN,8575.0,9331.0,10351.0,9842.0,9927.0,10863.0,...,Lincoln Park,Lincoln Park,"Sun., Closed; Mon. & Wed., 10-6; Tue. & Thu., ...",1150 W. Fullerton Ave.,Chicago,IL,60614,(312) 744-1926,https://www.chipublib.org/locations/44/,"(41.925591694002954, -87.6581132450843)"
1,NaN,4602.0,6041.0,NaN,4950.0,4858.0,5008.0,6127.0,5987.0,4965.0,...,Kelly,Kelly,"Sun., Closed; Mon. & Wed., 10-6; Tue. & Thu., ...",6151 S. Normal Blvd.,Chicago,IL,60621,(312) 747-8418,https://www.chipublib.org/locations/40/,"(41.78217163315653, -87.63740786532117)"
2,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,Independence,Independence,"Sun., Closed; Mon. & Wed., 10-6; Tue. & Thu., ...",4024 N. Elston Ave.,Chicago,IL,60618,(312) 744-0900,https://www.chipublib.org/locations/37/,"(41.953903720729926, -87.71694255042644)"
3,NaN,5293.0,6733.0,NaN,4444.0,5861.0,5160.0,6162.0,6221.0,5621.0,...,King,King,"Sun., Closed; Mon. & Wed., 10-6; Tue. & Thu., ...",3436 S. King Dr.,Chicago,IL,60616,(312) 747-7543,https://www.chipublib.org/locations/41/,"(41.83180139956726, -87.6177606656296)"
4,NaN,7495.0,8779.0,NaN,6958.0,7174.0,8181.0,7983.0,8318.0,7382.0,...,Coleman,Coleman,"Sun., Closed; Mon. & Wed., Noon-8; Tue. & Thu....",731 E. 63rd St.,Chicago,IL,60637,(312) 747-7760,https://www.chipublib.org/locations/22/,"(41.78037301095412, -87.60715724280014)"


In [235]:
# okay now let's make this a geodataframe
rel_lib_data = lib_data[['year', 'branch_name', 'YTD', 'LOCATION']]
rel_lib_data['geometry'] = rel_lib_data['LOCATION'].apply(swap_points).apply(Point)
lib_gdf = gpd.GeoDataFrame(rel_lib_data, geometry='geometry')
# merge with community area
lib_comm_df = gpd.sjoin(lib_gdf, comm_area_gdf, how="left", op='intersects')
lib_comm_df['year'] = lib_comm_df['year'].astype('int')
# merge with crime
lib_crime_df = pd.merge(lib_comm_df[['year', 'branch_name', 'YTD', 'community', 'community_area']],
                    crime_by_year, on=['community_area', 'year', 'community'], how='left')
lib_crime_df.head()

/Users/katykoeing/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,year,branch_name,YTD,community,community_area,crime_count
0,2017,Lincoln Park,119885.0,LINCOLN PARK,7,4005
1,2017,Kelly,58574.0,ENGLEWOOD,68,5728
2,2017,Independence,0.0,IRVING PARK,16,2865
3,2017,King,68071.0,DOUGLAS,35,2557
4,2017,Coleman,95022.0,WOODLAWN,42,3915


In [257]:
# add L stop info
rail_stops = pd.read_csv('data/L_stops.csv')
rail_stops['geometry'] = rail_stops['Location'].apply(swap_points).apply(Point)
rail_gdf = gpd.GeoDataFrame(rail_stops, geometry='geometry')
rail_comm = gpd.sjoin(rail_gdf, comm_area_gdf, how="left", op='intersects')
rail_grouped = rail_comm.groupby('community_area')['STATION_NAME'].nunique().\
                         reset_index().rename(columns={'STATION_NAME': 'num_stations'})
rail_grouped.head()

,community_area,num_stations
0,1,4
1,10,1
2,11,1
3,14,3
4,16,3


In [274]:
final_df = pd.merge(lib_crime_df, rail_grouped, on='community_area', how='left').fillna(0)
final_df['period'] = final_df['year'] - 2014
sides_df = pd.read_csv('community-areas.csv').drop(columns = 'Unnamed: 2')
sides_df['community_area'] = sides_df['id']
sides_df = sides_df.astype(str)
final_w_sides = pd.merge(final_df, sides_df[['community_area', 'zone']], on=['community_area'])
final_w_sides.head()

,year,branch_name,YTD,community,community_area,crime_count,num_stations,period,zone
0,2017,Lincoln Park,119885.0,LINCOLN PARK,7,4005,2.0,3,NORTH
1,2016,Lincoln Park,139486.0,LINCOLN PARK,7,3827,2.0,2,NORTH
2,2015,Lincoln Park,121396.0,LINCOLN PARK,7,3425,2.0,1,NORTH
3,2018,Lincoln Park,136011.0,LINCOLN PARK,7,4587,2.0,4,NORTH
4,2019,Lincoln Park,127570.0,LINCOLN PARK,7,3977,2.0,5,NORTH


In [275]:
final_w_sides.to_csv('lib_crime_station.csv', index=False)